In [ ]:
def clusteriza_sh4(imp_fert,n_cluster=4):
    
    imp_fert['log_kg']=np.log10(imp_fert.KG_LIQUIDO)
    imp_fert['log_fob']=np.log10(imp_fert.VL_FOB)
    
    from sklearn.preprocessing import MinMaxScaler as MMS
    MMS=MMS()
    
    imp_fert=imp_fert.assign(s_log_kg=MMS.fit_transform(imp_fert[['log_kg']]))
    imp_fert=imp_fert.assign(s_log_fob=MMS.fit_transform(imp_fert[['log_fob']]))
    
    from sklearn.cluster import KMeans as KM
    KM=KM(n_clusters=n_cluster)
    
    col_k_s=['s_log_kg','s_log_fob']
    
    q=pd.DataFrame()
    for sh4 in imp_fert.CO_SH4.unique():
        imp_fert_t=imp_fert[imp_fert.CO_SH4==sh4]
        imp_fert_t=imp_fert_t.assign(km=KM.fit_predict(imp_fert_t[col_k_s]))
        q=pd.concat([q,imp_fert_t])
    
    imp_fert=q.copy()
    
    # Renomeando os clusters de acordo com o rank entre eles
    q=pd.DataFrame()
    for sh4 in imp_fert.CO_SH4.unique():
        t=imp_fert[imp_fert.CO_SH4==sh4]
        t1=t.groupby(by=['CO_SH4','km']).KG_LIQUIDO.mean().reset_index().sort_values('KG_LIQUIDO')
        t1=t1.assign(km2=np.arange(0,len(t1)))
        q=pd.concat([q,t1])
    q=q.drop(columns='KG_LIQUIDO')
    imp_fert=imp_fert.merge(q)
    
    imp_fert=imp_fert.drop(columns='km').rename(columns={'km2':'km'})

    imp_fert=imp_fert.sort_values('nick_sh4')
    
    plt.show()

    return imp_fert

print('** Função clusteriza_sh4(imp_fert,n_cluster=4) disponível **')